In [1]:
import pandas as pd
import numpy as np

In [2]:
#load training dataset
#ratings_big = pd.read_csv('train_dt/ratings.csv') 
    # movies_metadata와 merge 한 후의 갯수 : 11428646
ratings = pd.read_csv('train_dt/ratings_small.csv')

movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')

credits = pd.read_csv('train_dt/credits.csv')

keywords = pd.read_csv('train_dt/keywords.csv')

links = pd.read_csv('train_dt/links.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_2864\1772567852.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')


movies_metadata processing

In [3]:
#나중에 최종결과 보여줄 때 movies_metadata_origin을 사용할 예정
movies_metadata_origin = movies_metadata

In [4]:
#계산에 불필요한 ft drop
movies_metadata.drop(['homepage','overview', 'tagline', 'poster_path'], axis = 1, inplace = True)

#이 두 ft는 kaggle에는 안뜨는 데 다운받으면 있음
#의미를 몰라서 일단 drop
movies_metadata.drop(['vote_average','vote_count'], axis = 1, inplace = True)

#movies_metadata_origin[movies_metadata_origin['production_companies'] == "[]"] -> 결과 : 11559 
#null값이 11559개(전체 movie dataset : 45031개)라 총 1/4정도를 차지해서 없애는 것이 낫다고 판단
movies_metadata.drop(['production_companies'], axis = 1, inplace = True)

In [5]:
#belongs_to_collection null 값을 no collection 값으로 변경
movies_metadata['belongs_to_collection'].replace(np.nan,'no collection', inplace = True)

In [6]:
#null값 없애기
movies_metadata.dropna(inplace = True)

In [7]:
#'genres', 'proudction_countries'가 null인 경우 drop
movies_metadata = movies_metadata.loc[movies_metadata_origin['genres'] != "[]"]
movies_metadata = movies_metadata.loc[movies_metadata_origin['production_countries'] != "[]"]

In [8]:
#movies_metadata에서 budget이 0 인 것 개수
len(movies_metadata.loc[movies_metadata['budget'] == '0']['budget'])

29600

In [9]:
movies_metadata['budget'] = pd.to_numeric(movies_metadata['budget'])

In [10]:
#movies_metadata['budget'] 0인 것은 나머지의 평균으로 넣는다.
budget_mean = movies_metadata.loc[movies_metadata['budget'] != 0]['budget'].mean()
movies_metadata['budget'].replace(0, budget_mean, inplace = True)

In [11]:
#'id'형식이 날짜 형식인 경우 -> 잘못된 경우
#ex) ValueError: Unable to parse string "2012-09-29" at position 29502
movies_metadata['isIdRight'] = movies_metadata['id'].str.contains('|'.join('-'))
movies_metadata = movies_metadata[movies_metadata['isIdRight'] == False]
movies_metadata.drop(['isIdRight'], axis = 1, inplace = True)

In [12]:
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
movies_metadata.loc['id'] = movies_metadata['id'].astype('str')
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'])

credits preprocessing

In [23]:
#credits의 [] - null값 제거
credits = credits.loc[credits['cast'] != "[]"]
credits = credits.loc[credits['crew'] != "[]"]

In [24]:
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
credits['id'] = pd.to_numeric(credits['id'])

merging for making complete training dataset

In [26]:
all_movie_df = pd.merge(movies_metadata, credits)

In [27]:
all_movie_df

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",3.000000e+07,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,tt0114709,en,Toy Story,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,False,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,no collection,6.500000e+07,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,tt0113497,en,Jumanji,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,False,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",2.232716e+07,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,tt0113228,en,Grumpier Old Men,11.7129,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,False,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,no collection,1.600000e+07,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,tt0114885,en,Waiting to Exhale,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,False,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",2.232716e+07,"[{'id': 35, 'name': 'Comedy'}]",11862.0,tt0113041,en,Father of the Bride Part II,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,False,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37232,False,no collection,2.232716e+07,"[{'id': 27, 'name': 'Horror'}]",289923.0,tt0252966,en,The Burkittsville 7,0.38645,"[{'iso_3166_1': 'US', 'name': 'United States o...",2000-10-03,0.0,30.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Burkittsville 7,False,"[{'cast_id': 2, 'character': 'Branwall', 'cred...","[{'credit_id': '5403d669c3a3682d9800427d', 'de..."
37233,False,no collection,2.232716e+07,"[{'id': 878, 'name': 'Science Fiction'}]",222848.0,tt0112613,en,Caged Heat 3000,0.661558,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Caged Heat 3000,False,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de..."
37234,False,no collection,2.232716e+07,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840.0,tt0102797,en,Robin Hood,5.683753,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Robin Hood,False,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de..."
37235,False,no collection,2.232716e+07,"[{'id': 18, 'name': 'Drama'}]",111109.0,tt2028550,tl,Siglo ng Pagluluwal,0.178241,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,Century of Birthing,False,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."


In [28]:
all_movie_df.isnull().sum()

adult                    0
belongs_to_collection    0
budget                   0
genres                   0
id                       0
imdb_id                  0
original_language        0
original_title           0
popularity               0
production_countries     0
release_date             0
revenue                  0
runtime                  0
spoken_languages         0
status                   0
title                    0
video                    0
cast                     0
crew                     0
dtype: int64

In [ ]:
ratings['movieId'] = ratings['movieId'].astype('str')
new_df = ratings.merge(movies_metadata, how = 'left', left_on = 'movieId', right_on = 'id')

In [ ]:
print(len(new_df))
new_df.head(5)

In [ ]:
new_df.dropna(inplace = True)

In [ ]:
len(new_df)

In [ ]:
new_df.head(5)

In [ ]:
len(ratings)

In [ ]:
len(movies_metadata)